In [1]:
! pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import dateutil.parser as parser
from bs4 import BeautifulSoup


In [75]:
SCOPES = ['https://mail.google.com/','https://www.googleapis.com/auth/gmail.modify']

creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())


#label_id_one = 'UNREAD'
label_id_one = 'INBOX'
label_id_one = 'CATEGORY_UPDATES'
service = build('gmail', 'v1', credentials=creds)
unread_msgs = service.users().messages().list(userId='me',labelIds=[label_id_one]).execute()
mssg_list = unread_msgs['messages']
print("get '"+label_id_one + "' list")

get 'CATEGORY_UPDATES' list


In [76]:
final_list = [ ]    
for mssg in mssg_list:
    temp_dict = { }
    m_id = mssg['id'] # get id of individual message
    message = service.users().messages().get(userId='me', id=m_id).execute() # fetch the message using API
    payld = message['payload'] # get payload of the message 
    headr = payld['headers'] # get header of the payload
    for header in headr:
        name = header['name']
        value = header['value']
        
        if name == 'From':
            temp_dict['Sender'] = value
        elif name == 'Subject':
            temp_dict['Subject'] = value
        elif name == 'Date':
            date_parse = parser.parse(value)
            m_date = date_parse.date()
            temp_dict['Date'] = str(m_date)
        temp_dict['msg_id'] = m_id
        '''
        try:
            # Fetching message body
            mssg_parts = payld['parts'] # fetching the message parts
            part_one  = mssg_parts[0] # fetching first element of the part 
            part_body = part_one['body'] # fetching body of the message
            part_data = part_body['data'] # fetching data from the body
            clean_one = part_data.replace("-","+") # decoding from Base64 to UTF-8
            clean_one = clean_one.replace("_","/") # decoding from Base64 to UTF-8
            clean_two = base64.b64decode (bytes(clean_one, 'UTF-8')) # decoding from Base64 to UTF-8
            soup = BeautifulSoup(clean_two , "lxml" )
            mssg_body = soup.body()
            # mssg_body is a readible form of message body
            # depending on the end user's requirements, it can be further cleaned 
            # using regex, beautiful soup, or any other method
            temp_dict['Message_body'] = mssg_body
        except :
            pass'''
        
    final_list.append(temp_dict)
print ("temp_dict strlen: ", str(len(final_list)))

temp_dict strlen:  100


In [53]:
final_list

[{'msg_id': '18a9cc017e75b4c4',
  'Date': '2023-09-16',
  'Subject': '安全性快訊',
  'Sender': 'Google <no-reply@accounts.google.com>'},
 {'msg_id': '18a9c1a8d62018cf',
  'Date': '2023-09-16',
  'Subject': '安全性快訊',
  'Sender': 'Google <no-reply@accounts.google.com>'},
 {'msg_id': '18a9b7763ddf0aed',
  'Date': '2023-09-15',
  'Sender': 'epassbook@tdcc.com.tw',
  'Subject': '集保e手掌握銀行資訊更新次數彙整通知'},
 {'msg_id': '18a9b6841a5e62a3',
  'Date': '2023-09-16',
  'Sender': '"台北富邦行動銀行" <mbank@dfm.taipeifubon.com.tw>',
  'Subject': '台北富邦行動銀行一般登入交易「成功」通知'},
 {'msg_id': '18a9b673b09df092',
  'Date': '2023-09-16',
  'Sender': 'Apple <News@insideapple.apple.com>',
  'Subject': '<AD> 立即預訂 iPhone 15 Pro 與 iPhone 15。'},
 {'msg_id': '18a9b018a39a5849',
  'Date': '2023-09-16',
  'Sender': '"玉山銀行" <Service@info.esunbank.com>',
  'Subject': '玉山網路銀行每日帳務訊息通知'},
 {'msg_id': '18a9933066bc15e1',
  'Subject': '【課程即將上線】填問卷領 500 元課程折扣券，體驗會好評達 4.9 顆星🔥',
  'Sender': '"富果團隊" <service@fugle.tw>',
  'Date': '2023-09-15'},
 {'ms

In [78]:
import pandas as pd
df=pd.DataFrame(final_list)
print(df.shape)
df

(100, 4)


,msg_id,Date,Sender,Subject
0,18a9b7763ddf0aed,2023-09-15,epassbook@tdcc.com.tw,集保e手掌握銀行資訊更新次數彙整通知
1,18a9b673b09df092,2023-09-16,Apple <News@insideapple.apple.com>,<AD> 立即預訂 iPhone 15 Pro 與 iPhone 15。
2,18a9b018a39a5849,2023-09-16,"""玉山銀行"" <Service@info.esunbank.com>",玉山網路銀行每日帳務訊息通知
3,18a9933066bc15e1,2023-09-15,"""富果團隊"" <service@fugle.tw>",【課程即將上線】填問卷領 500 元課程折扣券，體驗會好評達 4.9 顆星🔥
4,18a9925ecae4b8bc,2023-09-15,Firstrade Securities <service@firstrade.com>,Top Options Trading Ideas for the Coming Week
...,...,...,...,...
95,1898647ff955358a,2023-07-24,"""新加坡商蝦皮娛樂電商有限公司台灣分公司"" <b2ceci@ecimail3.tradeva...",新加坡商蝦皮娛樂電商有限公司台灣分公司電子發票開立通知
96,18985a6fa6433448,2023-07-24,"""104人力銀行"" <jobbank@104.com.tw>",【台達電子工業股份有限公司 _DELTA ELECTRONICS INC.】發給您的【詢問意願】
97,1898359ddb1a0a48,2023-07-23,"""定錨產業筆記"" <postmaster@mailer.investanchors.com>",【定錨產業筆記】台股觀察週報 2023/7/23
98,18982f8409a759f5,2023-07-23,"""蝦皮購物"" <info@shopee.tw>",您的貨到付款訂單#230723QTDNQ8XV已被確認


In [79]:
#result = df[df['Sender'].str.contains('shopee')]
keywords = ['linkedin', 
            #'shopee',
            #'104',
            'info@shopee.tw'
            'service@ms3.hncb.com.tw',
            'no-reply@accounts.google.com',
            '財經M平方',
            'epassbook@tdcc.com.tw', #集保中心
            'dailydozen@email.forbes.com',#富比世週報
            'account@seekingalpha.com',#seek alpha
            'billhunter@billhunter.kgieworld.com.tw',#凱基期貨
            'mail@mail.adobe.com',#adobe
            'edm@momo.com.tw',#momo廣告
            'alumni@nchu.edu.tw',#中興大學校友報馬仔
            'PERSOLKELLY',#推工作的
            '方格子',
            'noreply@medium.com'
           ]
pattern = '|'.join(keywords) 
result = df[df['Sender'].str.contains(pattern, case=False, regex=True)]
print(result.shape)
result

(8, 4)


,msg_id,Date,Sender,Subject
0,18a9b7763ddf0aed,2023-09-15,epassbook@tdcc.com.tw,集保e手掌握銀行資訊更新次數彙整通知
5,18a98c823441b66b,2023-09-15,"""MacroMicro.me 財經M平方"" <web@mg.macromicro.me>",【MM團隊來信】湯傑策，和你分享M平方里程碑，送上一份專屬禮物 🎁
6,18a97bc314d50f9c,2023-09-15,Google <no-reply@accounts.google.com>,已成功取回您的 Google 帳戶
7,18a97bc30cd26273,2023-09-15,Google <no-reply@accounts.google.com>,針對 jaytang9876@gmail.com 發出的安全性警示
12,18a9697f7ff5f9d7,2023-09-15,Google <no-reply@accounts.google.com>,針對 jaytang9876@gmail.com 發出的安全性警示
15,18a9614517ab433c,2023-09-14,Google <no-reply@accounts.google.com>,安全性快訊
21,18a85b11bc65c5c3,2023-09-12,"""集保e手掌握"" <epassbook@tdcc.com.tw>",【集保】「集保e手掌握」App新增以電子方式接收股利發放通知功能，請貴客戶儘速完成eNoti...
57,189e34f6cd78e984,2023-08-10,Google Accounts Team <no-reply@accounts.google...,更新《閒置 Google 帳戶政策》


In [80]:
for msg_id in result['msg_id']:
    #label_id = 'Notes'  # 不能用要找出對應Label_5
    #label_body = { "addLabelIds": ["UNREAD","Label_5"]}
    label_body = { "addLabelIds": ["Label_5"]}#python專屬
    #label_body =  {"removeLabelIds": ["UNREAD"]}
    print(msg_id)
    # 使用 Gmail API 添加標  
    msg = service.users().messages().modify(userId='me', id=msg_id, body=label_body).execute()

18a9b7763ddf0aed
18a98c823441b66b
18a97bc314d50f9c
18a97bc30cd26273
18a9697f7ff5f9d7
18a9614517ab433c
18a85b11bc65c5c3
189e34f6cd78e984


In [82]:
keywords = ['驗證',
            '成功',
            '課程',
            '通知'
           ]
pattern = '|'.join(keywords)
result = df[df['Subject'].str.contains(pattern, case=False, regex=True)]
print(result.shape)
result

(24, 4)


,msg_id,Date,Sender,Subject
0,18a9b7763ddf0aed,2023-09-15,epassbook@tdcc.com.tw,集保e手掌握銀行資訊更新次數彙整通知
2,18a9b018a39a5849,2023-09-16,"""玉山銀行"" <Service@info.esunbank.com>",玉山網路銀行每日帳務訊息通知
3,18a9933066bc15e1,2023-09-15,"""富果團隊"" <service@fugle.tw>",【課程即將上線】填問卷領 500 元課程折扣券，體驗會好評達 4.9 顆星🔥
6,18a97bc314d50f9c,2023-09-15,Google <no-reply@accounts.google.com>,已成功取回您的 Google 帳戶
8,18a97bb6d0c88a01,2023-09-15,Google <noreply@google.com>,Google 驗證碼
10,18a97232165d8373,2023-09-15,"""玉山銀行"" <Service@info.esunbank.com>",e.Fingo會員權益及服務通知
20,18a8679ac2b02cbe,2023-09-11,Firstrade Securities <service@firstrade.com>,為您的賬戶啟用雙重身份驗證（2FA）
21,18a85b11bc65c5c3,2023-09-12,"""集保e手掌握"" <epassbook@tdcc.com.tw>",【集保】「集保e手掌握」App新增以電子方式接收股利發放通知功能，請貴客戶儘速完成eNoti...
27,18a6b0ff49a72c68,2023-09-06,einvoice@fia.gov.tw,財政部電子發票整合服​務平台-消費發票彙整通知，手機條碼:/MK**ROR(每期)
31,18a44f98c861f642,2023-08-30,"""Hahow - 好學校"" <course-message@hahow.in>",課程「動畫互動網頁程式入門 (HTML/CSS/JS)」發佈了一篇新公告「墨雨互動設計 課程...


In [72]:
for msg_id in result['msg_id']:
    label_body = { "addLabelIds": ["Label_5"]}#python專屬
    #label_body =  {"removeLabelIds": ["UNREAD"]}
    print(msg_id)
    # 使用 Gmail API 添加標  
    msg = service.users().messages().modify(userId='me', id=msg_id, body=label_body).execute()

18a9b7763ddf0aed
18a9b6841a5e62a3
18a9b018a39a5849
18a9933066bc15e1
18a97bc314d50f9c
18a97bb6d0c88a01
18a973407052f291
18a97232165d8373
18a8679ac2b02cbe
18a85b11bc65c5c3
18a6b0ff49a72c68
18a67f238f8ce028
18a44f98c861f642
18a0dfd493889892
189ef5db56d7ccdd
189ef5053e7ce032
189ef1a2238c5a43
189deaeaa06f26ee
189deae276b5b1d5
189cb29801ef58ba
189cb271a13dec90
189aea0895fa6837
189a9849c01d266f
189a2727c25f5a6c
189905bf3ead0204
1898908be7f3ae65
